# 2D flying-snake at Re=2000 and AoA=35deg (exact markers)

In [1]:
import os
import sys
import collections

import ipywidgets

import snake
from snake import miscellaneous
from snake.petibm.simulation import PetIBMSimulation
from snake.cuibm.simulation import CuIBMSimulation

In [2]:
print('Python version: ' + sys.version)
print('snake version:' + snake.__version__)

Python version: 2.7.11 |Anaconda 2.5.0 (64-bit)| (default, Dec  6 2015, 18:08:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
snake version:0.1


---

### Description

Flying-snake at Reynolds number 2000 and angle-of-attack 35 degrees.

We found that the coordinates of the Lagrangian markers were not identical to those used in Krishnan et al. (2014).
In our previous simulations with PetIBM, the boundary was slightly shifted over a disctance that is less than 1 cell-width (< 0.004).
This happened because we rotated the geometry around its center of mass while Krishnan et al. (2014) did the rotation around the origin (0,0).

We would like to see how, this displacement in the Lagragian markers affects the numerical solution.

---

### General information

* software: `PetIBM`
* version: branch `master`, tag `snake-repro-2`
* dependencies: `PETSc-3.5.2`, `OpenMPI-1.8`, and `GCC-4.9.2`
* submission date: 2016/04/26 (mesnardo)
* running date: 2016/04/27
* machine: `Colonial One` (short queue, 32 processes, 2 nodes)

In [ ]:
simulation = PetIBMSimulation(description='PetIBM')

---

### Computational domain and mesh

We use the same input file than Krishnan et al. (2014).

In [ ]:
domain_info = os.path.join(simulation.directory, 'cartesianMesh.yaml')
%cat $domain_info

---

### Boundary conditions

We use the same input file than Krishnan et al. (2014).

In [ ]:
boundary_info = os.path.join(simulation.directory, 'flowDescription.yaml')
%cat $boundary_info

---

### Simulation parameters

In [ ]:
simulation_info = os.path.join(simulation.directory, 'simulationParameters.yaml')
%cat $simulation_info
solvers_info = os.path.join(simulation.directory, 'solversPetscOptions.info')
%cat $solvers_info

---

### Running the case

This simulation was run on our university HPC cluster, Colonial One.

To run the simulation (from the simulation directory):
    > mpirun -n <nprocs> <path to petibm2d executable> \
            -directory <simulation directory> \
            -log_summary

---

### Post-processing

We computed 80 non-dimensional time-units in 37.5 hours.

#### Forces

In [ ]:
simulation.read_forces()
simulation.get_mean_forces(limits=[32.0, 64.0])
simulation.get_strouhal(limits=[32.0, 64.0], order=200)

simulation.plot_forces(display_coefficients=True, 
                       coefficient=2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       style='snakeReproducibility',
                       save_name='forceCoefficients')
dataframe = simulation.create_dataframe_forces(display_strouhal=True,
                                               display_coefficients=True,
                                               coefficient=2.0)

In [ ]:
krishnan = CuIBMSimulation(description='Krishnan et al. (2014)')
krishnan.read_forces(file_path=os.path.join(os.environ['SNAKE'],
                                            'resources',
                                            'flyingSnake2d_cuibm_anush',
                                            'flyingSnake2dRe2000AoA35',
                                            'forces'))
krishnan.get_mean_forces(limits=[32.0, 64.0])
krishnan.get_strouhal(limits=[32.0, 64.0], order=200)

simulation.plot_forces(display_coefficients=True, 
                       coefficient=2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=krishnan, 
                       other_coefficients=2.0,
                       style='snakeReproducibility',
                       save_name='forceCoefficientsCompareKrishnanEtAl2014')
dataframe2 = krishnan.create_dataframe_forces(display_strouhal=True,
                                              display_coefficients=True,
                                              coefficient=2.0)

In [ ]:
other = PetIBMSimulation(description='PetIBM (shifted markers)',
                         directory=os.path.relpath('../../shiftedMarkers/Re2000AoA35',
                                                   simulation.directory))
other.read_forces()
other.get_mean_forces(limits=[32.0, 64.0])
other.get_strouhal(limits=[32.0, 64.0], order=200)

simulation.plot_forces(display_coefficients=True,
                       coefficient=2.0,
                       display_extrema=True, order=200,
                       limits=(0.0, 80.0, 0.0, 3.0),
                       other_simulations=other,
                       other_coefficients=2.0,
                       save_name='forceCoefficientsCompareShiftedMarkers')
dataframe3 = other.create_dataframe_forces(display_strouhal=True,
                                           display_coefficients=True,
                                           coefficient=2.0)

In [ ]:
dataframes = dataframe.append([dataframe2, dataframe3])
print(dataframes)

In [ ]:
images_directory = os.path.join(simulation.directory,
                                'images')
figures = collections.OrderedDict()
figures['present'] = os.path.join(images_directory,
                                  'forceCoefficients.png')
figures['present + Krishnan et al. (2014)'] = os.path.join(images_directory,
                                                           'forceCoefficientsCompareKrishnanEtAl2014.png')
figures['present + shifted markers'] = os.path.join(images_directory,
                                                    'forceCoefficientsCompareShiftedMarkers.png')
ipywidgets.interact(miscellaneous.display_image, figure=figures);

#### Vorticity field

In [ ]:
simulation.read_grid()
time_steps = simulation.get_time_steps()
for time_step in time_steps:
    simulation.read_fields('vorticity', time_step)
    simulation.plot_contour('vorticity',
                            field_range=[-5.0, 5.0, 101],
                            filled_contour=True,
                            view=[-2.0, -5.0, 15.0, 5.0],
                            width=8.0,
                            time_increment=0.0004,
                            style='snakeReproducibility')

In [ ]:
miscellaneous.displayer(os.path.join(simulation.directory, 
                                     'images', 
                                     'vorticity_-2.00_-5.00_15.00_5.00'))